**Carregando dataset**

In [2]:
import pandas as pd

dataSemStopWords = pd.read_csv('../../datasets/dataSemStopWords.csv', engine='python')


In [ ]:
import spacy

In [ ]:
pip install tensorflow

**Tokenização**

* Antes de alimentar nosso  LSTM, precisamos converter o texto em vetores numéricos de tamanho fixo. O código a seguir executa um pipeline completo: primeiro, um Tokenizer do Keras é usado para construir um vocabulário com as 15.000 palavras mais frequentes e transformar nossas avaliações em sequências de números.

* Em seguida, como as redes neurais exigem entradas de comprimento uniforme, aplicamos o Padding para garantir que todas as sequências tenham exatamente 150 tokens, além de dividir o treino/teste.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=15000, oov_token="<unk>")  # Limitar o vocabulário aos 15000 termos mais frequentes

tokenizer.fit_on_texts(dataSemStopWords['texto_lematizado'])

sequences = tokenizer.texts_to_sequences(dataSemStopWords['texto_lematizado'])

max_length = 150  # Tamanho máximo da sequência (ajuste conforme necessidade)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# 6. Dividir em conjuntos de treino e teste
from sklearn.model_selection import train_test_split

X = padded_sequences
y = dataSemStopWords['feedback']  # Sua variável alvo

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Tamanho do vocabulário: 42997
Forma do array de sequências: (132373, 150)
Exemplo de sequência original: [6, 466, 3448, 15, 40, 3219, 59, 14, 1288, 28, 126]...
Exemplo de sequência padronizada: [   6  466 3448   15   40 3219   59   14 1288   28  126    0    0    0
    0    0    0    0    0    0]...


**Class Weights**

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Pega as classes únicas presentes nos seus dados de treino
classes = np.unique(y_train)

# Calcula os pesos. O modo 'balanced' faz o cálculo automaticamente.
pesos = compute_class_weight(class_weight='balanced',
                             classes=classes,
                             y=y_train)

# O primeiro valor é o peso da classe 0, o segundo da classe 1, etc.
print(f"Classes: {classes}")
print(f"Pesos calculados: {pesos}")

Classes: [0 1 2]
Pesos calculados: [1.23398355 2.70451527 0.54949149]


In [ ]:
pip install imbalanced-learn


**Construindo o Modelo LSTM**

* Para esta tarefa de classificação de sentimento, irei construir um modelo de rede neural sequencial. A arquitetura começa com uma camada de Embedding, que é responsável por transformar os números inteiros do nosso vocabulário em vetores densos de 128 dimensões. É nesta camada que o modelo aprenderá as relações de significado entre as palavras.

* O núcleo do modelo é uma camada Bidirectional LSTM com 64 unidades. A LSTM (Long Short-Term Memory) é projetada para entender padrões em sequências, e o fato de ser bidirecional permite que ela analise o texto tanto da esquerda para a direita quanto da direita para a esquerda. Isso fornece um contexto muito mais rico de cada palavra, melhorando a capacidade do modelo de capturar a intenção do texto.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

# Definir dimensões e hiperparâmetros
vocab_size = min(len(tokenizer.word_index) + 1, 15000)  # +1 para índice de padding (0)
embedding_dim = 128  # Dimensionalidade dos embeddings
max_length = 150  # Comprimento das sequências (definido anteriormente)

# Construir o modelo
model = Sequential([
    # Camada de Embedding
    Embedding(vocab_size, embedding_dim),

    # Primeira camada bidirecional LSTM
    Bidirectional(LSTM(64)),
    Dropout(0.4),

    # Camadas Dense (totalmente conectadas)
    Dense(64, activation='relu'),
    Dropout(0.5),

    # Camada de saída (3 classes: negativo, neutro, positivo)
    Dense(3, activation='softmax')
])

**Preparando o dicionário dos pesos**

In [ ]:
class_weight_dict = {i: weight for i, weight in enumerate(pesos)}

**Compilando o modelo com ADAM como optimizador**
* Escolhi ele pois é um algoritmo robusto e amplamente utilizado que ajusta os pesos da rede de forma eficiente para minimizar os erros

In [ ]:

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)



**Treinamento com Callbacks**
* Com o modelo compilado, iniciarei a fase de treinamento. Para evitar overfitting e garantir que será salvo a versão mais performática do nosso modelo, utilizarei callbacks, que irá monitorar o processo de treino a cada época.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 1. Configurar o ModelCheckpoint para salvar o melhor modelo
model_checkpoint = ModelCheckpoint(
    filepath='melhor_modelo.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min'
)

# 2. Configurar o EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=2,
    verbose=1,
    mode='min'
)

# 3. Treinar com AMBOS os callbacks na lista
history = model.fit(
    X_train_res, y_train_res,
    validation_data=(X_test, y_test),
    epochs=15,
    batch_size=64,
    class_weight=class_weight_dict,
    callbacks=[early_stopping, model_checkpoint] # <--- Use a lista com os dois
)

# 4. (Passo Opcional, mas recomendado) Carregar o melhor modelo salvo
# Após o treinamento, o objeto 'model' em memória será o da última época.
# Para garantir que você está usando o melhor modelo, carregue-o do arquivo.
from tensorflow.keras.models import load_model

print("\nCarregando o melhor modelo salvo...")
model = load_model('melhor_modelo.keras')
print("Melhor modelo carregado com sucesso.")


Epoch 1/15
3012/3012 [==============================] - 312s 102ms/step - loss: 0.7576 - accuracy: 0.6403 - val_loss: 0.6093 - val_accuracy: 0.7643
Epoch 2/15
1321/3012 [============>.................] - ETA: 2:43 - loss: 0.6310 - accuracy: 0.7043

KeyboardInterrupt: 

**Métricas**

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         1920000   
                                                                 
 bidirectional (Bidirection  (None, 128)               98816     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                        

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Supondo que 'X_test_pad' e 'y_test' são seus dados de teste já processados e padronizados

print("\n--- Avaliação Final no Conjunto de Teste ---")

# 1. Obter a perda (loss) e a acurácia
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Loss no Teste: {loss:.4f}")
print(f"Acurácia no Teste: {accuracy * 100:.2f}%")

# 2. Fazer predições para obter o relatório detalhado
y_pred_probs = model.predict(X_test)

_classes = np.argmax(y_pred_probs, axis=1) # Pega o índice da classe com maior probabilidade

# 3. Gerar Relatório de Classificação e Matriz de Confusão
print("\nRelatório de Classificação:")
print(classification_report(y_test, _classes, target_names=['Negativo', 'Neutro', 'Positivo']))

print("\nMatriz de Confusão:")
cm = confusion_matrix(y_test, _classes)
# Visualizar matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Negativo', 'Neutro', 'Positivo'],
            yticklabels=['Negativo', 'Neutro', 'Positivo'])
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.title('Matriz de Confusão')
plt.show()


--- Avaliação Final no Conjunto de Teste ---


KeyboardInterrupt: 

In [ ]:
# Opção 1: Use o texto original através do tokenizador (reverso)
textos_originais = []
for seq in X_test:
    texto = tokenizer.sequences_to_texts([seq])[0]
    textos_originais.append(texto)

df_analise = pd.DataFrame({'texto_tokenizado': textos_originais})

# Adicionar as colunas com os rótulos verdadeiros e as previsões
df_analise['real_label'] = y_test
df_analise['predicted_label'] = _classes

# Adicionar colunas com os nomes das classes
mapa_classes = {0: 'Negativo', 1: 'Neutro', 2: 'Positivo'}
df_analise['real_class_name'] = df_analise['real_label'].map(mapa_classes)
df_analise['predicted_class_name'] = df_analise['predicted_label'].map(mapa_classes)

# Verificar como ficou o DataFrame
print("DataFrame de Análise criado:")
display(df_analise.head())

NameError: name '_classes' is not defined

In [ ]:
# Condição 1: O rótulo real era Neutro (índice 1)
condicao_real_neutro = df_analise[df_analise['real_label'] == 1]

# Condição 2: O rótulo previsto foi Positivo (índice 2)
condicao_previsto_positivo = condicao_real_neutro[condicao_real_neutro['predicted_label'] == 1]

condicao_previsto_positivo.shape


In [ ]:
condicao_previsto_positivo.head(30)


In [ ]:
# Condição 1: O rótulo real era Neutro (índice 1)
condicao_real_neutro = df_analise[df_analise['real_label'] == 1]

# Condição 2: O rótulo previsto foi Positivo (índice 2)
condicao_previsto_positivo2 = condicao_real_neutro[condicao_real_neutro['predicted_label'] == 2]

condicao_previsto_positivo2.shape

In [ ]:
condicao_previsto_positivo2.head(30)